# E KNN and K-means CIFAR
_5 points_

transform cifar-10 to grayscale. 

- Does knn work similarly good?
- Does k-means work similarly good?
- Demonstrate this similar to  B, C and D <br><br>

## Solution

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score
import skimage as ski
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
plt.gray()

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
# transform to grayscale
x_train_bw = ski.color.rgb2gray(x_train)
x_test_bw = ski.color.rgb2gray(x_test)

# flatten the grayscale dataset
x_train_flat = x_train_bw.reshape([x_train.shape[0],
                              x_train.shape[1] * x_train.shape[2]])
x_test_flat = x_test_bw.reshape([x_test.shape[0],
                           x_test.shape[1] * x_test.shape[2]])
# flattening the y_arrays
y_train = y_train.ravel()
y_test = y_test.ravel()
x_train_bw.shape

(50000, 32, 32)

In [5]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

### Testing Knn

In [6]:
knn_2 = KNeighborsClassifier(n_neighbors=2, n_jobs=-1)
knn_4 = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
knn_8 = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)

In [7]:
# subsampling with 10000 images
knn_2.fit(x_train_flat[:10000], y_train[:10000])
knn_4.fit(x_train_flat[:10000], y_train[:10000])
knn_8.fit(x_train_flat[:10000], y_train[:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=8, p=2,
           weights='uniform')

In [8]:
knn_2_predictions = knn_2.predict(x_test_flat[:2000])
knn_4_predictions = knn_4.predict(x_test_flat[:2000])
knn_8_predictions = knn_8.predict(x_test_flat[:2000])

In [9]:
# As in the previous examples we will use the error rate as 1 - f1_score for all models

knn_2_f1_score = f1_score(y_test[:2000], knn_2_predictions, average="micro")
knn_4_f1_score = f1_score(y_test[:2000], knn_4_predictions, average="micro")
knn_8_f1_score = f1_score(y_test[:2000], knn_8_predictions, average="micro")
print("Error rate for knn {}: {}".format(2, (1 - knn_2_f1_score)))
print("Error rate for knn {}: {}".format(4, (1 - knn_4_f1_score)))
print("Error rate for knn {}: {}".format(8, (1 - knn_8_f1_score)))

Error rate for knn 2: 0.798
Error rate for knn 4: 0.772
Error rate for knn 8: 0.7685


In [10]:
# Creating the individual error rates for each label using the f1_score

prediction_array = [knn_2_predictions, knn_4_predictions, knn_8_predictions]
results = [] # will later contain the results of all three models
for i in range(len(prediction_array)):
    predictions = prediction_array[i]
    result_array = []
    for i in range(10):
        indices = np.where(y_test[:2000] ==i)[0]
        true_values = y_test[indices] # shorten the y_test array to the specified indices
        pred_values = predictions[indices]
        result_array.append(1 - f1_score(true_values, pred_values, average="micro"))
    results.append(result_array)
print("Error rates by model and label")
knn_df = pd.DataFrame(np.array(results).T, columns=["Knn_2", "Knn_4", "Knn_8"], index=class_names)
knn_df

Error rates by model and label


,Knn_2,Knn_4,Knn_8
airplane,0.505102,0.612245,0.637755
automobile,0.818182,0.833333,0.924242
bird,0.666667,0.646154,0.610256
cat,0.849246,0.899497,0.909548
deer,0.797980,0.707071,0.676768
dog,0.935135,0.935135,0.940541
frog,0.907407,0.800926,0.787037
horse,0.891192,0.875648,0.870466
ship,0.645161,0.516129,0.460829
truck,0.970443,0.921182,0.901478


### Testing Kmeans

In [11]:
kmeans_10_iter = KMeans(n_clusters=10, max_iter=10, n_jobs=-1).fit(x_train_flat[:10000]) # n_jobs = -1 to use all cores for calculation
kmeans_100_iter = KMeans(n_clusters=10, max_iter=100, n_jobs=-1).fit(x_train_flat[:10000])
kmeans_1000_iter = KMeans(n_clusters=10, max_iter=1000, n_jobs=-1).fit(x_train_flat[:10000])

In [12]:
# visualize how many iterations were actually made
print(kmeans_10_iter.n_iter_)
print(kmeans_100_iter.n_iter_)
print(kmeans_1000_iter.n_iter_)

9
69
64


In [13]:
kmeans_10_scores = []
kmeans_100_scores = []
kmeans_1000_scores = []

# create empty multidimensional arrays used for better indexing
for i in range(10):
    kmeans_10_scores.append([])
    kmeans_100_scores.append([])
    kmeans_1000_scores.append([])

# filling the score_arrays (lists) with the corresponding true labels (y_test)
#kmeans_10
count = 0
for value in kmeans_10_iter.labels_:
    kmeans_10_scores[value].append(y_train[count])
    count += 1

#kmeans_100
count = 0
for value in kmeans_100_iter.labels_:
    kmeans_100_scores[value].append(y_train[count])
    count += 1

#kmeans_1000
count = 0
for value in kmeans_1000_iter.labels_:
    kmeans_1000_scores[value].append(y_train[count])
    count += 1

In [14]:
# calculate a distribution between the ten classes
kmeans_10_clusters = []
kmeans_100_clusters = []
kmeans_1000_clusters = []
for i in range(10):
    # 10 iterations
    local_array = np.bincount(kmeans_10_scores[:][i])
    kmeans_10_clusters.append(local_array)
    # 100 iterations
    local_array = np.bincount(kmeans_100_scores[:][i])
    kmeans_100_clusters.append(local_array)
    #1000 iterations
    local_array = np.bincount(kmeans_1000_scores[:][i])
    kmeans_1000_clusters.append(local_array)

In [15]:
# Calculating the majority label in each cluster and its percentage
print("Kmeans with 10 iterations\n")
count = 0
labels = np.arange(10)
majority_labels_10 = []
percentages_maj_cluster_10 = []
for array in kmeans_10_clusters:
    maximum = np.where(array == max(array))[0]
    if len(maximum) == 1:
        print("Cluster: {} => majority label: {}".format(count, maximum[0]))    
        majority_labels_10.append(maximum[0])
        percentages_maj_cluster_10.append(max(array) / sum(array))
    else: # catch the case that the maximum exists more than once in a cluster
        print("Multiple maximum detected! No value has been added to the result set!")
    count += 1
differences_10_iter = np.where(np.isin(labels, majority_labels_10) == False)[0]
print("The following numbers have no own cluster: {}".format(differences_10_iter))
#print(percentages_maj_cluster_10)

Kmeans with 10 iterations

Cluster: 0 => majority label: 4
Multiple maximum detected! No value has been added to the result set!
Cluster: 2 => majority label: 7
Cluster: 3 => majority label: 8
Cluster: 4 => majority label: 0
Cluster: 5 => majority label: 8
Cluster: 6 => majority label: 5
Cluster: 7 => majority label: 6
Cluster: 8 => majority label: 2
Cluster: 9 => majority label: 9
The following numbers have no own cluster: [1 3]


In [16]:
print("Kmeans with 100 iterations\n")
count = 0
labels = np.arange(10)
majority_labels_100 = []
percentages_maj_cluster_100 = []
for array in kmeans_100_clusters:
    maximum = np.where(array == max(array))[0]
    if len(maximum) == 1:
        print("Cluster: {} => majority label: {}".format(count, maximum[0]))    
        majority_labels_100.append(maximum[0])
        percentages_maj_cluster_100.append(max(array) / sum(array))
    else:
        print("Multiple maximum detected! No value has been added to the result set!")
    count += 1
differences_100_iter = np.where(np.isin(labels, majority_labels_100) == False)[0]
print("The following numbers have no own cluster: {}".format(differences_100_iter))
#print(percentages_maj_cluster_100)

Kmeans with 100 iterations

Cluster: 0 => majority label: 9
Cluster: 1 => majority label: 0
Cluster: 2 => majority label: 2
Cluster: 3 => majority label: 4
Cluster: 4 => majority label: 7
Cluster: 5 => majority label: 9
Cluster: 6 => majority label: 8
Cluster: 7 => majority label: 6
Cluster: 8 => majority label: 8
Cluster: 9 => majority label: 6
The following numbers have no own cluster: [1 3 5]


In [17]:
print("Kmeans with 1000 iterations\n")
count = 0
labels = np.arange(10)
majority_labels_1000 = []
percentages_maj_cluster_1000 = []
for array in kmeans_1000_clusters:
    maximum = np.where(array == max(array))[0]
    if len(maximum) == 1:
        print("Cluster: {} => majority label: {}".format(count, maximum[0]))    
        majority_labels_1000.append(maximum[0])
        percentages_maj_cluster_1000.append(max(array) / sum(array))
    else:
        print("Multiple maximum detected! No value has been added to the result set!")
    count += 1
differences_1000_iter = np.where(np.isin(labels, majority_labels_1000) == False)[0]
print("The following numbers have no own cluster: {}".format(differences_1000_iter))
#print(percentages_maj_cluster_1000)

Kmeans with 1000 iterations

Cluster: 0 => majority label: 9
Cluster: 1 => majority label: 8
Cluster: 2 => majority label: 9
Cluster: 3 => majority label: 4
Cluster: 4 => majority label: 2
Cluster: 5 => majority label: 6
Cluster: 6 => majority label: 7
Cluster: 7 => majority label: 0
Cluster: 8 => majority label: 6
Cluster: 9 => majority label: 8
The following numbers have no own cluster: [1 3 5]


In [46]:
combined_percentages = [percentages_maj_cluster_10, percentages_maj_cluster_100, percentages_maj_cluster_1000]
df0 = pd.DataFrame(combined_percentages)#, columns=["10", "100", "1000"])
df0.T

,0,1,2
0,0.209402,0.301136,0.305825
1,0.165523,0.333895,0.150897
2,0.280130,0.164662,0.188047
3,0.329966,0.191985,0.203636
4,0.256219,0.158046,0.163601
5,0.206831,0.182561,0.150168
6,0.216524,0.314249,0.142271
7,0.181818,0.155556,0.332186
8,0.351634,0.152876,0.228321
9,NaN,0.227216,0.315789


### Task D: What is difficult to predict?

In [ ]:
# create a new knn model
knn_3 = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn_3.fit(x_train_flat[:10000], y_train[:10000])
knn_3_predictions = knn_3.predict(x_test_flat[:2000])

In [ ]:
# f1 score of the knn model
print(f1_score(y_test[:2000], predictions[:2000], average="micro"))

In [ ]:
# error contains all indices where the predictions do not match the real labels
error = []
count = 0
for value in knn_3_predictions:
    if value != y_test[count]:
        error.append(count)
    count += 1
#error = np.where(knn_3_predictions == y_test)
print(knn_3_predictions[:20])
print(y_test[:20])
#print(error)
# looking at example images that were not identified correctly
for value in error[:25]:
    x_val = value
    print("Prediction: {}".format(class_names[knn_3.predict([x_test_bw[x_val].reshape(32*32)])[0]]))
    print("Real value: {}".format(class_names[y_test[x_val]]))
    plt.imshow(x_test_bw[x_val])
    plt.show()
    
#print("Knn Labels not recognized: {}".format(np.bincount(y_test[error])))
#print("Knn Labels wrongly classified: {}".format(np.bincount(predictions[error])))

In [ ]:
print("What is shown in these pictures?")
arr = [6, 17, 21]
strings = ["cat", "deer", "deer"]
for i in arr:
    print("Real value: {}".format(strings.pop(0)))
    plt.imshow(x_test_bw[error[i]])
    plt.show()

# Answer

## Evaluating similarities on knn and kmeans compared to the MNIST dataset

In comparison to the MNIST dataset, the CIPHER10 dataset has very bad results. While the error rate for all three Knns on the MNIST dataset was around 3%, the CIPHER10 dataset has errors around 76-79%.<br>
Also the kmeans model did not perform better because more categories have no own cluster and the individual percentages in the clusters decreased, meaning the predicted values inside a cluster are close next to each other leading to many false predicitons.<br>
At this point we have to say that the CIPHER10 dataset was only used as a subsample because of runtime issues (10k training, 2k test). The results might differ if the whole dataset is used.

## Ex. B for CIPHER10

The CIPHER10 has the following error rates for the different knn models:<br>
Error rate for knn 2: 0.798<br>
Error rate for knn 4: 0.772<br>
Error rate for knn 8: 0.7685<br>

The individual error rates can be seen below. There, one can see that especially the most classes perform very bad, so we want to highlight the ones that were not completely false. For knn = 2 the class airplane performed best with around 50% false predictions; for knn = 4 the class ship was around 50% failure and for knn = 8 the class ship has below 50% failure.<br>
All other classes have higher error rates, going up to 97% (knn = 2 class: truck).

In contrast to the MNIST dataset, one can see a solid difference by changing the value for k. Firstly, with a rising k (in our example from 2 to 4 to 8) the overall error rate decreases; nevertheless it is a very horrible score.
Furthermore, with a rising k the errors in some classes constantly increase or decrease; for example increasing errors for rising k in classes "cat" and "dog" and decreasing errors for rising k in "bird", "deer" and "ship".

In [ ]:
knn_df

## Ex. C for CIPHER10

Majority classes for each cluster:<br>
<b>For max_iterations = 10</b><br>
Cluster: 0 => majority label: 9
Cluster: 1 => majority label: 5
Cluster: 2 => majority label: 8
Cluster: 3 => majority label: 2
Cluster: 4 => majority label: 6
Cluster: 5 => majority label: 6
Cluster: 6 => majority label: 8
Cluster: 7 => majority label: 7
Cluster: 8 => majority label: 0
Cluster: 9 => majority label: 4<br><br>

Majority classes for each cluster:<br>
<b>For max_iterations = 100</b><br>
Cluster: 0 => majority label: 7
Cluster: 1 => majority label: 9
Cluster: 2 => majority label: 8
Cluster: 3 => majority label: 2
Cluster: 4 => majority label: 9
Cluster: 5 => majority label: 6
Cluster: 6 => majority label: 6
Cluster: 7 => majority label: 4
Cluster: 8 => majority label: 0
Cluster: 9 => majority label: 8<br><br>

Majority classes for each cluster:<br>
<b>For max_iterations = 1000</b><br>
Cluster: 0 => majority label: 7
Cluster: 1 => majority label: 6
Cluster: 2 => majority label: 8
Cluster: 3 => majority label: 2
Cluster: 4 => majority label: 8
Cluster: 5 => majority label: 9
Cluster: 6 => majority label: 0
Cluster: 7 => majority label: 4
Cluster: 8 => majority label: 6
Cluster: 9 => majority label: 9<br><br>

<b>Categories that have no own cluster:</b><br>
max_iter=10: Classes 1 and 3<br>
max_iter=100: Classes 1, 3 and 5<br>
max_iter=1000: Classes 1, 3 and 5<br><br>

### Majority percentage in the cluster (table below)
One can see that the classes in many clusters have to be very even distributed, if the maximum class only reaches percentages around 13-19%. On the other hand, in some classes the maximum category is far stronger (>30%); however, compared to the MNIST dataset where the maximum categories reached up to 90% and never fell below 35% this values are a good proof that the Kmeans model is not very effective for this task/dataset.

In [ ]:
df0

## Ex. D for CIPHER10

In general, one can say that no class is predicted with a good accuracy. But, as seen above at the images, some classes are worse to predict than others. Problems occur especially with the animal classes, for example dogs and cats are often not recognized and mislabeled. <br>
The performance for trucks and especially ships on the other hand are far better; however, regarding the absolute values and comparing it to the MNIST dataset the errors are still very high.<br>
Finally, we can say that this dataset is harder to predict than the MNIST dataset. This might be because the classes are more complicated; the pictures are a little bigger and transforming the images to grayscale might take important features out of the pictures which could be recognized in another model.